Committing to a branch in Git is super straightforward! Here's how it works step-by-step:

---

### ✅ **1. Make sure you're on the right branch**

Check your current branch:
```bash
git branch
```
If you're not on the branch you want to commit to (e.g., `feature/login`), switch to it:

```bash
git checkout feature/login
```

---

### ✅ **2. Make changes to files**

Edit or add files. For example:

```bash
echo "console.log('login page');" > Application/login.js
```

---

### ✅ **3. Stage your changes**

You need to "stage" the changes before committing:

```bash
git add .
```

Or stage a specific file:

```bash
git add Application/login.js
```

---

### ✅ **4. Commit your changes**

Now create a commit with a descriptive message:

```bash
git commit -m "Add login page functionality"
```

This commit is now part of the **current branch only** — it does *not* affect `main` or any other branches.

---

### 🔄 **(Optional) 5. Push to remote branch (e.g., GitHub)**

```bash
git push origin feature/login
```

If it's the first time pushing this branch:

```bash
git push -u origin feature/login
```

---


----
**Note**

### The file downloads, but the browser can’t **decode or contains 0 frames**

From the screenshot you showed earlier the `<video>` element appears, shows
**0 s / 0 s** and refuses to play.  
That always means one of two things:

| Cause | Symptom | Why it happens in your code |
|-------|---------|-----------------------------|
| **A. The MP4 contains a codec the browser can’t play** | DevTools ▶ Console: “Media resource could not be decoded.” | OpenCV writes **MPEG‑4 Part 2 (`mp4v`)**. Modern Chrome / Firefox / Edge only accept **H.264 (`avc1`)** or VP9. |
| **B. The MP4 is practically empty (0 frames)** | File size only a few kB; timeline 0 s / 0 s | `cap.read()` is failing (couldn’t decode the upload), so the `while` loop never writes a frame. |

Let’s fix both at once.

---

## 1 . Use whatever **working codec** your OpenCV build has

The wheel on Raspberry Pi OS usually has **MJPEG**, sometimes **H.264**, but
not VP9.  Try the list in order and stop at the first that opens:

```python
candidates = [("avc1", ".mp4"), ("H264", ".mp4"), ("X264", ".mp4"),
              ("MJPG", ".avi")]          # MJPEG always available

writer = None
for fourcc_str, ext in candidates:
    fourcc = cv2.VideoWriter_fourcc(*fourcc_str)
    tmp_out_path = tempfile.mktemp(suffix=ext)
    w = cv2.VideoWriter(tmp_out_path, fourcc, fps, (width, height))
    if w.isOpened():
        writer, out_path = w, tmp_out_path
        mime = "video/mp4" if ext == ".mp4" else "video/x-msvideo"
        break
```

If `writer is None` → decoding failed; return `{"error": "no frames found"}`.

**Browsers can play H.264 and MJPEG AVI**; the latter is large but always
works.

---

## 2 . Make sure you **actually read frames**

When the upload is an H.264 clip but your OpenCV lacks that decoder, these two
lines return **zero** – every time:

```python
ok, frame = cap.read()      # ok == False
cap.get(cv2.CAP_PROP_FPS)   # fps == 0
```

Add a guard:

```python
if fps == 0 or width == 0 or height == 0:
    cap.release()
    os.remove(tmp_in_path)
    return {"error": "Unsupported video codec"}
```

and show that error as a toast in the front‑end.

---

## 3 . Front‑end: autoplay & clear previous file

```javascript
const vid = document.createElement('video');
vid.controls = true;
vid.autoplay = true;          // start right away
vid.src = url;
$('#fileCard').appendChild(vid);
```

---

### Quick checklist

1. **DevTools ▶ Network**: `Content‑Type` is `video/mp4` or `video/x‑msvideo`.
2. **Size** in the Response tab is more than a few kB.
3. **Console** has no `MediaError`.
4. **If still 0 s / 0 s** → OpenCV never decoded the upload. Install full
   FFmpeg (`sudo apt install ffmpeg`) or convert client‑side.

With the codec‑probing block above the Pi 5 will at least return MJPEG AVI,
which every browser will play, and your `<video>` element will show the
annotated clip instead of a stuck placeholder.

Here’s a step‑by‑step for creating a self‑signed TLS cert with **openssl**, serving it from your Pi, and then installing & trusting it on your iPhone:

---

## 1) Generate a self‑signed certificate on the Pi

Open a shell on your Pi and run (replace **192.168.1.42** with your Pi’s LAN IP or hostname):

```bash
# 1. Create a 2048‑bit RSA key + self‑signed cert, valid 1 year:
openssl req -x509 -nodes -days 365 \
  -newkey rsa:2048 \
  -keyout pi-selfsigned.key \
  -out pi-selfsigned.crt \
  -subj "/CN=192.168.1.42"
```

You’ll end up with two files in your current folder:
- `pi-selfsigned.key`  → the private key  
- `pi-selfsigned.crt`  → the certificate

---

## 2) Convert the certificate to DER format for iOS

iOS expects DER or “.cer” containers. Convert your PEM `.crt` into DER:

```bash
openssl x509 -in pi-selfsigned.crt -outform der -out pi-selfsigned.cer
```

Now you have `pi-selfsigned.cer`, which you’ll install on the iPhone.

---

## 3) Serve the `.cer` so your iPhone can fetch it

A quick way is to drop `pi-selfsigned.cer` into your FastAPI `static/` folder (e.g. `templates/static/`) and expose it:

- URL becomes `https://192.168.1.42:8000/static/pi-selfsigned.cer`

*(Don’t worry that it’s over HTTPS—even if the cert isn’t trusted yet, Safari will let you download it.)*

---

## 4) Run Uvicorn with your self‑signed cert

Start your app pointing to the key + cert:

```bash
uvicorn main:app \
  --host 0.0.0.0 \
  --port 8000 \
  --ssl-keyfile templates/static/pi-selfsigned.key \
  --ssl-certfile templates/static/pi-selfsigned.crt
```

Now your server listens on `https://192.168.1.42:8000`.

---

## 5) Install & trust the cert on your iPhone

1. **Fetch the cert**  
   On your iPhone’s Safari, visit:  
   ```
   https://192.168.1.125:8000/static/pi-selfsigned.cer
   ```  
   Safari will download a “Profile”.

2. **Install the profile**  
   - When prompted, tap **Allow**.  
   - Open **Settings** → **Profile Downloaded** (near the top).  
   - Tap **pi-selfsigned.cer**, then **Install** (you may need to enter your passcode).

3. **Trust the certificate**  
   - In Settings go to **General** → **About** → **Certificate Trust Settings**.  
   - Under **ENABLE FULL TRUST FOR ROOT CERTIFICATES**, flip **ON** for your “pi-selfsigned.cer”.

4. **Confirm**  
   - Now browse to `https://192.168.1.42:8000/` in Safari.  
   - You should see the lock icon with no “Not Secure” warning, and `getUserMedia()` will work.

---

### Notes

- This profile installation is **per‑device**; you’ll need to repeat on each iPhone/iPad you use.  
- The cert is valid for 365 days (as per `-days 365`); after that you’ll need to regenerate.  
- If you change your Pi’s IP or hostname, regenerate the cert with the new **CN**.

With this in place, your page will be served over HTTPS that iOS actually trusts, so `getUserMedia()` and WebRTC will work seamlessly on your iPhone.